In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
train = pd.read_csv('/Users/hajiaga/Downloads/house-prices-advanced-regression-techniques/train.csv')


In [4]:
train.head()

NameError: name 'train' is not defined